In [1]:
print("heloooooooooooo, mic testing, mic testing")

heloooooooooooo, mic testing, mic testing


In [2]:
#setting ENVs 
import os
import getpass
#importing the env variables
os.environ["COHERE_API_KEY"]=os.getenv("COHERE_API_KEY")

#LANGSMITH TRACKING
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [3]:
from langchain_community.document_loaders import PyPDFLoader # type: ignore
loader = PyPDFLoader("attention.pdf")
pages  = loader.load()

In [4]:
pages[0]

Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing w

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter # type: ignore
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20)
text_splitter.split_documents(pages)[:5]

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='reproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='scholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='Google Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='Google Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗', metadata={'source': 'attention.pdf', 'page': 0})]

In [6]:
documents = text_splitter.split_documents(pages)
documents


[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='reproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='scholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='Google Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='Google Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='Llion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_cont

In [7]:
#importing Cohere Embeddings and LanceDB VectorDB 

from langchain_cohere import CohereEmbeddings # type: ignore
from langchain.vectorstores import LanceDB # type: ignore

embeddings = CohereEmbeddings()
docsearch = LanceDB.from_documents(documents, embeddings)

In [8]:
query="An attention function can be described as mapping a query "
result = docsearch.similarity_search(query)
result[0].page_content

'An attention function can be described as mapping a query and a set of key-value pairs to an'

Introduction to chains

In [10]:
#importing Cohere LLM 
from langchain_cohere import ChatCohere # type: ignore
from langchain_core.messages import HumanMessage # type: ignore
llm = ChatCohere(model="command", max_tokens=256, temperature=0.75) 
llm

ChatCohere(client=<cohere.client.Client object at 0x000002573804BD10>, async_client=<cohere.client.AsyncClient object at 0x0000025738060F90>, model='command', temperature=0.75, cohere_api_key=SecretStr('**********'))

In [12]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [14]:
retriever  = docsearch.as_retriever()
retriever

VectorStoreRetriever(tags=['LanceDB', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.lancedb.LanceDB object at 0x000002572ED95210>)

In [15]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)



In [16]:
response = retrieval_chain.invoke({"input": "Scaled Dot-Product Attention"})


In [17]:
response['answer']

'Based on the provided context, it can be understood that the Scaled Dot-Product Attention is a specific type of attention algorithm. Here is a detailed answer about the Scaled Dot-Product Attention algorithm: \n\nThe scaled dot-product attention is a type of attention that operates on a query, key, and value set. It computes a normalized similarity score between the query and key, where the values are multiplied by a scaling factor, to ensure that attention probabilities are appropriately calibrated. It helps enhance the representation of the query vector by attending to different key vectors, where attention ensures that the important key vectors are amplified and the less important ones are attenuated. \n\nThis attention process is a critical component in many different machine learning models such as the Transformer model.'